In [1]:
import os
import tensorflow as tf
import json
import pandas as pd
from os import getenv
#from keras import models
from firebase_admin import credentials, firestore, initialize_app

In [2]:
cred = credentials.Certificate("key.json")
default_app = initialize_app(cred)
db = firestore.client()

In [3]:
class User(object):
    def __init__(self, USER_ID, Nama, Gender, Age, Tipe_Masalah):
        self.USER_ID = USER_ID
        self.Nama = Nama
        self.Gender = Gender
        self.Age = Age
        self.Tipe_Masalah = Tipe_Masalah

    @staticmethod
    def from_json_do_not_use(source):
        """
        Source is expected from pure json string
        """
        return source

    @staticmethod
    def from_dict(source):
        # [START_EXCLUDE]
        user = User(source[u'USER_ID'], source[u'Nama'], source[u'Gender'], source[u'Age'], source[u'Tipe_Masalah'])

        if u'USER_ID' in source:
            user.USER_ID = source[u'USER_ID']

        if u'Nama' in source:
            user.Nama = source[u'Nama']

        if u'Gender' in source:
            user.Gender = source[u'Gender']

        if u'Age' in source:
            user.Age = source[u'Age']

        if u'Tipe_Masalah' in source:
            user.Tipe_Masalah = source[u'Tipe_Masalah']

        return user
        # [END_EXCLUDE]

    def to_dict(self):
        # [START_EXCLUDE]
        dest = {
            u'USER_ID': self.USER_ID,
            u'Nama': self.Nama,
            u'Gender': self.Gender,
            u'Age': self.Age,
            u'Tipe_Masalah': self.Tipe_Masalah
        }

        if self.USER_ID:
            dest[u'USER_ID'] = self.USER_ID

        if self.Nama:
            dest[u'Nama'] = self.Nama

        if self.Gender:
            dest[u'Gender'] = self.Gender

        if self.Age:
            dest[u'Age'] = self.Age

        if self.Tipe_Masalah:
            dest[u'Tipe_Masalah'] = self.Tipe_Masalah

        return dest
        # [END_EXCLUDE]

    
    def __repr__(self):
        return (
            f'City(\
                USER_ID={self.USER_ID}, \
                Nama={self.Nama}, \
                Age={self.Age}, \
                Gender={self.Gender}, \
                Tipe_Masalah={self.Tipe_Masalah}\
            )'
        )

In [4]:
zidan = User(1, "zidan", "pria", 43, "Depresi")

In [5]:
zidan

City(                USER_ID=1,                 Nama=zidan,                 Age=43,                 Gender=pria,                 Tipe_Masalah=Depresi            )

In [6]:
type(zidan)

__main__.User

In [7]:
type(zidan.to_dict())

dict

In [8]:
import pathlib
import os

user_file = "\\Data\\Processed User Data.json"
caregiver_file = "\\Data\\Processed Caregiver Data.json"

path = pathlib.Path().resolve()
parent_path = path.parent.absolute()

user_path = str(parent_path) + user_file
caregiver_ = str(parent_path) + caregiver_file

In [9]:

with open(user_path) as file:
    data = json.load(file)
    db_user_placeholder = db.collection(u'User_Data_Placeholder')
    for user_dict in data:
        user_dict['Nama'] = 'placeholder_nama'
        user_dict['Text'] = 'Placeholder_text'
        db_user_placeholder.add(user_dict)

In [10]:
data = db.collection(u'User_Data_Placeholder').get()


In [20]:
def convert_snapshot(data, desired_key):
    """
    this extract desired keys from list of dicts from snapshot
    """
    data_dictionary = map(lambda x: x.to_dict(), data)
    x = map(lambda x: dict((k, x[k]) for k in desired_key if k in x) , list(data_dictionary))
    
    return list(x)

In [21]:
new_data = convert_snapshot(data, ["Age","Tipe_Masalah","Gender","Text"])

In [22]:
print(new_data)

[{'Age': 27, 'Tipe_Masalah': 'Gangguan-kecemasan', 'Gender': 'Pria', 'Text': 'Placeholder_text'}, {'Age': 22, 'Tipe_Masalah': 'Gangguan-makan ADHD-Hiperaktif-dan-kurang-fokus', 'Gender': 'Pria', 'Text': 'Placeholder_text'}, {'Age': 44, 'Tipe_Masalah': 'Skizofrenia Depresi', 'Gender': 'Perempuan', 'Text': 'Placeholder_text'}, {'Age': 34, 'Tipe_Masalah': 'Skizofrenia', 'Gender': 'Perempuan', 'Text': 'Placeholder_text'}, {'Age': 44, 'Tipe_Masalah': 'Depresi', 'Gender': 'Perempuan', 'Text': 'Placeholder_text'}, {'Age': 37, 'Tipe_Masalah': 'Gangguan-stres-pascatrauma', 'Gender': 'Pria', 'Text': 'Placeholder_text'}, {'Age': 27, 'Tipe_Masalah': 'Depresi', 'Gender': 'Pria', 'Text': 'Placeholder_text'}, {'Age': 22, 'Tipe_Masalah': 'Gangguan-stres-pascatrauma', 'Gender': 'Pria', 'Text': 'Placeholder_text'}, {'Age': 39, 'Tipe_Masalah': 'Depresi Gangguan-kecemasan', 'Gender': 'Pria', 'Text': 'Placeholder_text'}, {'Age': 40, 'Tipe_Masalah': 'Gangguan-stres-pascatrauma ADHD-Hiperaktif-dan-kurang-fok

In [23]:
ds_kalimat = tf.data.Dataset.from_tensor_slices(pd.DataFrame.from_dict(new_data).to_dict(orient="list"))

ValueError: Can't convert Python sequence with mixed types to Tensor.

In [ ]:

def df_to_dataset(dataframe, shuffle=True, batch_size=32):
    """
    A utility method to create a tf.data dataset from a Pandas Dataframe
    """
    dataframe = dataframe.copy()
    ds = tf.data.Dataset.from_tensor_slices(dict(dataframe))
    if shuffle:
        ds = ds.shuffle(buffer_size=len(dataframe))
    ds = ds.batch(batch_size)
    return ds

In [34]:
def convert_categorical_data(df, col='Tipe_Masalah'):
    """
    Convert categorical data that splited by space to their own column in pandas dataframe , making them like one hot encoded
    """
    import numpy as np
    ### Join every string in every row, split the result, pull out the unique values.
    genres = np.unique(' '.join(df[col]).split(' '))
    ### Drop 'NA'
    genres = np.delete(genres, np.where(genres == ''))
    if(col=='Tipe_Masalah'):
        for genre in genres:
            df[genre] = df[col].str.contains(genre).astype('int')
    else:
        for genre in genres:
            df['Caregiver-'+genre] = df[col].str.contains(genre).astype('int')
    df.drop(col, axis=1, inplace=True)

In [35]:
user_ds = pd.DataFrame.from_dict(new_data)

In [38]:
user_ds.head(4)

,Age,Gender,Text,ADHD-Hiperaktif-dan-kurang-fokus,Depresi,Gangguan-kecemasan,Gangguan-makan,Gangguan-stres-pascatrauma,Skizofrenia
0,27,Pria,Placeholder_text,0,0,1,0,0,0
1,22,Pria,Placeholder_text,1,0,0,1,0,0
2,44,Perempuan,Placeholder_text,0,1,0,0,0,1
3,34,Perempuan,Placeholder_text,0,0,0,0,0,1


In [37]:
convert_categorical_data(user_ds, col='Tipe_Masalah')

In [ ]:
user_ds.head(4)